In [1]:
from tqdm.contrib.concurrent import process_map

from notebooks.ford.asbuilt import AsBuiltData
from notebooks.ford.decode import print_breakdown, search

# TODO: handle non-US cars
df_nhtsa = await search(
  include_openpilot=True,
  include_police=True,
  skip_missing_asbuilt=True,
)

# pre-load asbuilt
process_map(AsBuiltData.from_vin, df_nhtsa['VIN'].unique(), desc='Loading AsBuilt Data', chunksize=100)

print()
print_breakdown(df_nhtsa, include_model_year=False)

Loaded 63532 VINs (filter_comment=None, include_openpilot=True, skipped=168, missing_asbuilt=0)


Loading AsBuilt Data:   0%|          | 0/63532 [00:00<?, ?it/s]


Model
                       2
Aviator             1597
Bronco               997
Bronco Sport        3283
C-Max                  5
Continental           23
Corsair             1559
E-Transit            178
Ecosport             406
Edge                6722
Escape              9619
Expedition           835
Expedition MAX       733
Explorer            9422
F-150              14583
F-150 Lightning      442
F-250               1720
F-350               1158
F-450                207
F-550                  1
Fiesta               196
Flex                 189
Focus                169
Fusion              1163
GT                     3
MKC                   55
MKT                    7
MKZ                  100
Maverick            1356
Mustang              983
Mustang Mach-E       961
Nautilus            1974
Navigator            334
Navigator L          247
Ranger               533
Taurus               122
Transit             1234
Transit Connect      414
dtype: int64


In [2]:
df = df_nhtsa[['VIN', 'Make', 'Model', 'ModelYear']].copy()
df['CarName'] = df['Make'] + ' ' + df['Model'] + ' ' + df['ModelYear'].astype(str)
df.drop(columns=['Make', 'Model', 'ModelYear'], inplace=True)
df.head()

,VIN,CarName
0,MAJ6S3KL3NC467793,FORD Ecosport 2022
1,1FTEW1EP9KKC56452,FORD F-150 2019
2,1FTEW1EP3LFB56129,FORD F-150 2020
3,1FTEW1EP6MFB92964,FORD F-150 2021
4,1FTFW1E83PKD20196,FORD F-150 2023


In [3]:
import pandas as pd

from panda.python.uds import DATA_IDENTIFIER_TYPE
from notebooks.ford.ecu import FordEcu

ecu_map = {
  'ABS': FordEcu.AntiLockBrakeSystem,
  'APIM': FordEcu.AccessoryProtocolInterfaceModule,
  'IPMA': FordEcu.ImageProcessingModuleA,
  'PSCM': FordEcu.PowerSteeringControlModule,
}

def get_ecu_platform_code(abd: AsBuiltData, ecu: FordEcu) -> str | None:
  fw = abd.get_identifier(ecu, DATA_IDENTIFIER_TYPE.VEHICLE_MANUFACTURER_ECU_SOFTWARE_NUMBER)
  if fw is None:
    return None
  prefix, core, _ = fw.split('-')
  # core doesn't help split any platforms apart
  # the first year of the prefix (somewhat related to the year) does help split model generations
  return prefix
  # return f'{core}-{prefix}'

def get_platform_codes(row) -> pd.Series:
  abd = AsBuiltData.from_vin(row['VIN'])
  values: dict[str, str | None] = {}
  for ecu_name, ecu in ecu_map.items():
    values[ecu_name] = get_ecu_platform_code(abd, ecu)
  return pd.Series(values)

df_fw = df.join(df.apply(get_platform_codes, axis=1)).drop(columns=['VIN']).drop_duplicates()
df_fw.head()

,CarName,ABS,APIM,IPMA,PSCM
0,FORD Ecosport 2022,GN15,1U5T,None,GN15
1,FORD F-150 2019,KL34,1U5T,KL3T,KL3V
2,FORD F-150 2020,KL34,1U5T,KL3T,KL3V
3,FORD F-150 2021,ML34,NU5T,ML3T,ML3V
4,FORD F-150 2023,PL34,PU5T,PJ6T,ML3V


In [4]:
from notebooks.utils.union import merge_sets


def group_by(df: pd.DataFrame, by: list[str]) -> None:
  car_groups = list()
  for group in df.groupby(by, dropna=False):
    cars = set(group[1]['CarName'].unique())
    car_groups.append(cars)

  return merge_sets(car_groups)

In [5]:
from notebooks.ford.platforms import find_openpilot_platform

for ecus in (
  # ('ABS', 'PSCM'), - combines escape and bronco sport
  # ('ABS', 'APIM'), - combines escape and bronco sport
  # ('IPMA', 'PSCM'),
  # ('IPMA', 'APIM'),
  # ('PSCM', 'IPMA', 'APIM'), - combines escape and bronco sport
  # ('ABS', 'PSCM', 'APIM'),  - combines escape and bronco sport
  # ('ABS', 'IPMA', 'APIM'),
  ('ABS', ),
  ('ABS', 'IPMA'),  # seperate Bronco Sport and Escape
  ('ABS', 'IPMA', 'PSCM'),  # splits out: 2019 Mustang (from 2020-23), 2024 Bronco (from 2021-23)
  ('ABS', 'IPMA', 'PSCM', 'APIM'),  # splits out: 2019 EcoSport (from 2020-22), 2024 Escape/Corsair (from 2023)
):
  car_groups = group_by(df_fw, list(ecus))
  print(', '.join(ecus))
  print(f'Found {len(car_groups)} distinct car groups')
  for group in car_groups:
    # platforms = {find_openpilot_platform(car) for car in group}
    # print(f'{len(platforms)} platforms: {", ".join(map(str, platforms)):<33}; {", ".join(group)}')
    print('\t' + ', '.join(group))
  print()

ABS
Found 50 distinct car groups
	FORD Escape 2015
	FORD Fiesta 2017, FORD Fiesta 2018, FORD Fiesta 2019
	FORD Focus 2017, FORD Focus 2018
	FORD Edge 2018
	FORD Taurus 2019, FORD Flex 2018, LINCOLN MKT 2019, FORD Flex 2019, FORD Taurus 2018
	FORD Transit 2019, FORD Transit 2018
	LINCOLN Continental 2019, LINCOLN Continental 2020
	FORD F-150 2016
	FORD Ecosport 2020, FORD Ecosport 2022, FORD Ecosport 2021, FORD Ecosport 2018, FORD Ecosport 2019
	FORD Mustang 2018
	FORD Escape 2019, FORD Escape 2018, FORD Transit Connect 2018
	FORD C-Max 2018
	FORD Explorer 2017, FORD Explorer 2018, FORD Explorer 2019
	FORD F-250 2018, FORD F-350 2018, FORD F-450 2018
	FORD GT 2020
	FORD Fusion 2018
	FORD F-150 2018, FORD F-150 2017
	LINCOLN Navigator 2019, FORD Expedition MAX 2018, FORD Expedition 2019, FORD Expedition 2018, LINCOLN Navigator L 2019, FORD Expedition MAX 2019, LINCOLN  2019
	LINCOLN Nautilus 2019, FORD Edge 2019
	FORD Ranger 2021, FORD Ranger 2023, FORD Ranger 2019, FORD Ranger 2020, FOR

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /mnt/c/Users/camer/AppData/Local/Programs/Microsoft VS Code


In [6]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth', None)

df_reduced_ecus = df_fw.groupby(by=['ABS', 'IPMA'])[['CarName']].agg(set)
df_reduced_ecus['CarInfoPlatforms'] = df_reduced_ecus['CarName'].apply(lambda x: {find_openpilot_platform(car_name) for car_name in x})
df_reduced_ecus

CarName  \
ABS  IPMA                                                                                                                                                                                                                       
F1FC F1FT                                                                                                                                                                                                   {FORD Focus 2018}   
F2GC FL3T                                                                                                                                                                                                    {FORD Edge 2018}   
FG13 DA5T                                                                                                                                                              {FORD Taurus 2019, FORD Taurus 2018, LINCOLN MKT 2019}   
     EG1T                                                                                                                                                              {FORD Taurus 2019, FORD Taurus 2018, LINCOLN MKT 2019}   
FK41 CK4T                                                                                                                                                                              {FORD Transit 2019, FORD Transit 2018}   
G3GC GD9T                                                                                                                                                                {LINCOLN Continental 2019, LINCOLN Continental 2020}   
GV61 F1FT                                                                                                                                                                                {FORD Escape 2019, FORD Escape 2018}   
HB53 GB5T                                                                                                                                                                            {FORD Explorer 2018, FORD Explorer 2019}   
HC3C HC3T                                                                                                                                                                 {FORD F-250 2018, FORD F-350 2018, FORD F-450 2018}   
HG9C HS7T                                                                                                                                                                                                  {FORD Fusion 2018}   
HL34 HL3T                                                                                                                                                                                                   {FORD F-150 2018}   
JL14 JL1T                                                                  {FORD Expedition 2019, LINCOLN Navigator L 2019, FORD Expedition MAX 2019, LINCOLN Navigator 2019, FORD Expedition MAX 2018, FORD Expedition 2018}   
JL34 JL3T                                                                                                                                                                                                   {FORD F-150 2018}   
JR3C JR3T                                                                                                                                                                                                 {FORD Mustang 2018}   
K2GC KA1T                                                                                                                                                                                             {LINCOLN Nautilus 2019}   
     KT4T                                                                                                                                                                                                    {FORD Edge 2019}   
KB3C KB3T                                                                                                                          {FORD Ranger 2021, FORD Ranger 20

In [7]:
from collections import defaultdict
from typing import TypeVar

K, V = TypeVar('K'), TypeVar('V')

def default_dict_to_dict(d: defaultdict[K, V]) -> dict[K, V]:
  return {k: default_dict_to_dict(v) if isinstance(v, dict) else v for k, v in d.items()}

In [8]:
platforms_by_ecu_fw = defaultdict(lambda: defaultdict(set))

index_names = df_reduced_ecus.index.names
for index, row in df_reduced_ecus.iterrows():
  for platform in row['CarInfoPlatforms']:
    if platform is None:
      continue
    for ecu_name, ecu_fw in zip(index_names, index, strict=True):
      platforms_by_ecu_fw[ecu_name][ecu_fw].add(platform)

platforms_by_ecu_fw: dict[str, dict[str, set[str]]] = default_dict_to_dict(platforms_by_ecu_fw)
platforms_by_ecu_fw

{'ABS': {'K2GC': {<CAR.EDGE_MK2: 'FORD EDGE 2ND GEN'>},
  'L1MC': {<CAR.EXPLORER_MK6: 'FORD EXPLORER 6TH GEN'>},
  'L2GC': {<CAR.EDGE_MK2: 'FORD EDGE 2ND GEN'>},
  'LJ9C': {<CAR.MUSTANG_MACH_E_MK1: 'FORD MUSTANG MACH-E 1ST GEN'>},
  'LK9C': {<CAR.MUSTANG_MACH_E_MK1: 'FORD MUSTANG MACH-E 1ST GEN'>},
  'LX6C': {<CAR.BRONCO_SPORT_MK1: 'FORD BRONCO SPORT 1ST GEN'>,
   <CAR.ESCAPE_MK4: 'FORD ESCAPE 4TH GEN'>},
  'M2GC': {<CAR.EDGE_MK2: 'FORD EDGE 2ND GEN'>},
  'ML34': {<CAR.F_150_MK14: 'FORD F-150 14TH GEN'>},
  'ML3V': {<CAR.F_150_MK14: 'FORD F-150 14TH GEN'>},
  'N2GC': {<CAR.EDGE_MK2: 'FORD EDGE 2ND GEN'>},
  'NL34': {<CAR.F_150_MK14: 'FORD F-150 14TH GEN'>},
  'NL38': {<CAR.F_150_LIGHTNING_MK1: 'FORD F-150 LIGHTNING 1ST GEN'>},
  'NL3V': {<CAR.F_150_MK14: 'FORD F-150 14TH GEN'>},
  'NZ6C': {<CAR.MAVERICK_MK1: 'FORD MAVERICK 1ST GEN'>},
  'P2GC': {<CAR.EDGE_MK2: 'FORD EDGE 2ND GEN'>},
  'PL34': {<CAR.F_150_MK14: 'FORD F-150 14TH GEN'>},
  'PL38': {<CAR.F_150_LIGHTNING_MK1: 'FORD F-150 LI

In [9]:
for ecu_name, platforms_by_fw in platforms_by_ecu_fw.items():
  for fw, platforms in platforms_by_fw.items():
    if not platforms:
      continue
    print(f'{ecu_name:<6} prefix={fw}   {", ".join(platforms)}')

ABS    prefix=K2GC   FORD EDGE 2ND GEN
ABS    prefix=L1MC   FORD EXPLORER 6TH GEN
ABS    prefix=L2GC   FORD EDGE 2ND GEN
ABS    prefix=LJ9C   FORD MUSTANG MACH-E 1ST GEN
ABS    prefix=LK9C   FORD MUSTANG MACH-E 1ST GEN
ABS    prefix=LX6C   FORD BRONCO SPORT 1ST GEN, FORD ESCAPE 4TH GEN
ABS    prefix=M2GC   FORD EDGE 2ND GEN
ABS    prefix=ML34   FORD F-150 14TH GEN
ABS    prefix=ML3V   FORD F-150 14TH GEN
ABS    prefix=N2GC   FORD EDGE 2ND GEN
ABS    prefix=NL34   FORD F-150 14TH GEN
ABS    prefix=NL38   FORD F-150 LIGHTNING 1ST GEN
ABS    prefix=NL3V   FORD F-150 14TH GEN
ABS    prefix=NZ6C   FORD MAVERICK 1ST GEN
ABS    prefix=P2GC   FORD EDGE 2ND GEN
ABS    prefix=PL34   FORD F-150 14TH GEN
ABS    prefix=PL38   FORD F-150 LIGHTNING 1ST GEN
ABS    prefix=PL3V   FORD F-150 14TH GEN
ABS    prefix=PZ6C   FORD MAVERICK 1ST GEN
IPMA   prefix=KT4T   FORD EDGE 2ND GEN
IPMA   prefix=LB5T   FORD EXPLORER 6TH GEN
IPMA   prefix=LC5T   FORD EXPLORER 6TH GEN
IPMA   prefix=ML3T   FORD F-150 LIGHTNI

In [10]:
from openpilot.selfdrive.car.ford.values import CAR

SUPPORTED_PLATFORMS = {
  CAR.BRONCO_SPORT_MK1,
  CAR.ESCAPE_MK4,
  CAR.EXPLORER_MK6,
  CAR.F_150_MK14,
  CAR.F_150_LIGHTNING_MK1,
  CAR.MAVERICK_MK1,
  CAR.MUSTANG_MACH_E_MK1,
}

supported_platform_ecu_fws = defaultdict(lambda: defaultdict(set))

for ecu_name, platforms_by_fw in platforms_by_ecu_fw.items():
  for fw, platforms in platforms_by_fw.items():
    for platform in platforms & SUPPORTED_PLATFORMS:
      supported_platform_ecu_fws[platform][ecu_name].add(fw)

for platform, fw_by_ecu in supported_platform_ecu_fws.items():
  print(f'{platform:<30}: ({dict(fw_by_ecu)})')

FORD EXPLORER 6TH GEN         : ({'ABS': {'L1MC'}, 'IPMA': {'LC5T', 'LB5T'}})
FORD MUSTANG MACH-E 1ST GEN   : ({'ABS': {'LK9C', 'LJ9C'}, 'IPMA': {'RJ6T', 'ML3T', 'PJ6T'}})
FORD BRONCO SPORT 1ST GEN     : ({'ABS': {'LX6C'}, 'IPMA': {'M1PT'}})
FORD ESCAPE 4TH GEN           : ({'ABS': {'LX6C'}, 'IPMA': {'LJ6T'}})
FORD F-150 14TH GEN           : ({'ABS': {'PL34', 'PL3V', 'NL34', 'ML3V', 'NL3V', 'ML34'}, 'IPMA': {'RJ6T', 'ML3T', 'PJ6T'}})
FORD F-150 LIGHTNING 1ST GEN  : ({'ABS': {'PL38', 'NL38'}, 'IPMA': {'RJ6T', 'ML3T', 'PJ6T'}})
FORD MAVERICK 1ST GEN         : ({'ABS': {'NZ6C', 'PZ6C'}, 'IPMA': {'NZ6T'}})


In [11]:
def get_fw_prefix(fw: str) -> str:
  return fw.split('-')[0]

def try_fingerprint(row) -> set[str]:
  abs_fw, ipma_fw = row[['ABS', 'IPMA']]
  if abs_fw is None:
    return set()
  abs_fw_prefix = get_fw_prefix(abs_fw)
  abs_platforms = platforms_by_ecu_fw['ABS'].get(abs_fw_prefix, set())
  if len(abs_platforms) <= 1:
    return abs_platforms
  print(f'Multiple platforms after ABS: {abs_fw_prefix}: {abs_platforms}')

  if ipma_fw is None:
    return abs_platforms
  ipma_fw_prefix = get_fw_prefix(ipma_fw)
  ipma_platforms = platforms_by_ecu_fw['IPMA'].get(ipma_fw_prefix, set())

  platforms = abs_platforms & ipma_platforms
  if len(platforms) > 1:
    print(f'Multiple platforms after IPMA: {ipma_fw_prefix}: {ipma_platforms} (combined: {platforms})')
  return platforms


df_sample = df_fw[['CarName', 'ABS', 'IPMA']].sample(n=40).copy()
df_sample['Fingerprint'] = df_sample.apply(try_fingerprint, axis=1)
df_sample

,CarName,ABS,IPMA,Fingerprint
3,FORD F-150 2021,ML34,ML3T,{FORD F-150 14TH GEN}
169,FORD Escape 2018,GV61,None,{}
16330,FORD Bronco 2022,NB3V,M2DT,{}
2441,FORD Edge 2018,F2GC,None,{}
784,FORD Maverick 2024,PZ6C,NZ6T,{FORD MAVERICK 1ST GEN}
215,FORD Expedition 2021,ML14,LL1T,{}
1273,FORD F-350 2023,PC3C,PC3T,{}
2195,FORD Mustang Mach-E 2021,LK9C,RJ6T,{FORD MUSTANG MACH-E 1ST GEN}
1155,FORD Transit 2023,NK41,NK3T,{}
1015,FORD Maverick 2022,PZ6C,NZ6T,{FORD MAVERICK 1ST GEN}
